In [1]:
import os
import time
import cv2
import requests
import json
import numpy as np
import tensorflow as tf
from models.data_prepare import load_and_preprocess_image_pridict
from models.config import TABLE_PATH, TEST_PATH

## 配置信息

In [2]:
#字典
with open(TABLE_PATH, 'r',encoding='gbk') as f:
    inv_table = [char.strip() for char in f]+[' ']*2

In [3]:
#测试数据集
test_all_image_paths = [TEST_PATH + img for img in sorted(os.listdir(TEST_PATH))]

## 调用tensorflow serving方法

In [4]:
def text_ocr(image_list):
    '''
    矩形框文字识别
    '''
    result_list = []
    input_image = []
    max_length = max([int(32*img.shape[1]/img.shape[0]) for img in image_list])
    for img in image_list:
        image = cv2.resize(img, (int(32*img.shape[1]/img.shape[0]), 32), 3)
        image = image.astype('float32')
        image /= 255
        image -= 0.5
        image /= 0.5
        padding_im = np.zeros((32, max_length, 3), dtype=np.float32)
        padding_im[:, 0:image.shape[1],:] = image
        input_image.append(padding_im.tolist())
    url = 'http://192.168.46.230:7501/v1/models/crnn:predict'
    headers = {"content-type": "application/json"}
    data = json.dumps({
            "signature_name": 'serving_default',
            "instances":input_image})
    start = time.time()
    predictions = requests.post(url,data=data, headers=headers,timeout=50).json()
    print('识别时间：', time.time()-start)
    if 'predictions' in predictions:
        if len(predictions['predictions']) > 0:
            for sentence in predictions['predictions']:
                result = ''
                for i in range(len(sentence)):
                    char_index = np.argmax(sentence[i])
                    if char_index != len(inv_table):
                        result +=  inv_table[char_index]
                result_list.append(result)
    return result_list

## 准备图片

In [5]:
image_list = []
for i in range(81,85):
    image_list.append(cv2.imread(test_all_image_paths[i]))

## 调用接口

In [6]:
start = time.time()
result = text_ocr(image_list)
print('识别时间：', time.time()-start)
print('识别结果：', result)

识别时间： 16.232019424438477
识别时间： 16.373674392700195
识别结果： ['乙肝两对半定定量((番禺)', '检查 期 2019-12-30', '检查医生:迟琼', '项目名称']
